In [ ]:
import pulp

class LinearMultiObjectiveProblem:
    '''
    A linear multi-objective optimization problem defined by decision variables, constraints, and objective functions.
    LpMinimize is used as the optimization objective.
    '''
    def __init__(self, name, decision_variables, constraints, objective_functions):
        self.name = name
        self.objective_functions = objective_functions
        self.num_objectives = len(objective_functions)
        self.decision_variables = decision_variables
        self.constraints = constraints
        self.optimization_model = None
    
    def evaluate_objectives(self, decision_vector):
        return [func(decision_vector) for func in self.objective_functions]
    
    def build_optimization_model(self):
        self.optimization_model = pulp.LpProblem(self.name, pulp.LpMinimize)
        
        # Create decision variables
        lp_variables = {name: pulp.LpVariable(name, lowBound=low, upBound=up) 
                        for name, (low, up) in self.decision_variables.items()}
        
        # Add constraints
        for constraint in self.constraints:
            self.optimization_model += eval(constraint, {**lp_variables, **pulp.__dict__})
        
        # Set the first objective function as the optimization objective
        self.optimization_model += self.objective_functions[0]([lp_variables[v] for v in self.decision_variables])

    def solve_optimization_model(self):
        if not self.optimization_model:
            self.build_optimization_model()
        self.optimization_model.solve()
        return {v.name: v.varValue for v in self.optimization_model.variables()}

class SolutionVector:
    def __init__(self, problem, decision_vector):
        self.problem = problem
        self.decision_vector = decision_vector
        self._objective_values = None

    def get_objective_values(self):
        if self._objective_values is None:
            self._objective_values = self.problem.evaluate_objectives(self.decision_vector)
        return self._objective_values

    def __getattr__(self, name):
        if name.startswith('objective_') and name[10:].isdigit():
            index = int(name[10:]) - 1
            if 0 <= index < self.problem.num_objectives:
                return lambda: self.get_objective_values()[index]
        raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")




########################################################################################################################################
# Usage example
def f1(x):
    return 10 * x[0] + 1 * x[1]

def f2(x):
    return 1 * x[0] + 10 * x[1]

# Problem definition
decision_variables = {
    'x1': (0, None),  # x1 >=0
    'x2': (0, None)   # x2 >=0
}

problem_constraints = [
    "4*x1 + 1*x2 >= 8",           
    "1*x1 + 1*x2 >= 4",       
    "1*x1 + 8*x2 >= 8",
    "x1 + x2 <= 10"                   
]

objective_functions = [f1, f2]

# Create and solve the problem
myproblem = LinearMultiObjectiveProblem(
    "DummyLP", 
    decision_variables, 
    problem_constraints, 
    objective_functions
)

one_solution = myproblem.solve_optimization_model()
print("Optimal production plan:", one_solution)

# Create a solution with the optimal values
optimal_solution = SolutionVector(
    myproblem, 
    [one_solution['x1'], one_solution['x2']]
)

# Evaluate the solution for each objective function
print(f"Profit: {optimal_solution.objective_1()}")
print(f"Production: {optimal_solution.objective_2()}")

Optimal production plan: {'x1': 0.0, 'x2': 8.0}
Profit: 8.0
Production: 80.0
